In [299]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import uniform
import os
import time
import xgboost as xgb
from xgboost import XGBClassifier

from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

root = os.path.join(os.getcwd(), '..')

train_df = pd.read_csv(os.path.join(root, "data", "train.csv"))
test_df = pd.read_csv(os.path.join(root, "data", "test.csv"))

ocean = pd.read_csv(os.path.join(root, "data", "Bottle Field Descriptions.csv"), encoding = "ISO-8859-1")
meta = pd.read_csv(os.path.join(root, "data", "Cast Field Descriptions.csv"), encoding = "ISO-8859-1")

In [241]:
train_df_clean = (train_df
                  .drop(columns = ["Unnamed: 12", # junk col
                                   "Temperature_degC", # omit multicolinear cols
                                   "Salinity1", 
                                   "R_Nuts"])
                  .rename(columns = {"TA1.x": "TA1", # make col name name
                                     "R_Oxy_micromol.Kg": "R_Oxy_micromol_Kg"})) # clean name to prevent errors

In [254]:
# assign x and y
y_train = train_df_clean.iloc[:, -1]
x_train = train_df_clean.iloc[:, 1:(train_df_clean.shape[1] - 1)]

x_test = test_df.rename(columns = {"R_Oxy_micromol.Kg": "R_Oxy_micromol_Kg"}) # clean name to prevent errors

# scale predictors
scaler = StandardScaler()

# scale predictors
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test_scaled = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

In [243]:
train_df_clean

,id,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Oxy_micromol_Kg,PO4uM,SiO3uM,TA1,DIC
0,1,34.385030,-120.665530,0.030,33.80,0.00,7.79,323,141.2,0.642,37.40948,2.77,53.86,2287.45,2270.17
1,2,31.418333,-121.998333,0.000,34.70,0.00,7.12,323,140.8,0.767,64.81441,2.57,52.50,2279.10,2254.10
2,3,34.385030,-120.665530,0.180,14.20,0.00,11.68,50,246.8,0.144,180.29150,1.29,13.01,2230.80,2111.04
3,4,33.482580,-122.533070,0.013,29.67,0.01,8.33,232,158.5,0.562,89.62595,2.27,38.98,2265.85,2223.41
4,5,31.414320,-121.997670,0.000,33.10,0.05,7.53,323,143.4,0.740,60.03062,2.53,49.28,2278.49,2252.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1450,33.420000,-117.901667,0.000,38.60,0.00,6.52,520,118.2,0.889,12.17673,3.11,73.10,2313.52,2311.19
1450,1451,34.271667,-120.023333,0.330,5.90,0.00,13.21,49,290.1,0.149,224.81120,0.75,6.50,2223.31,2048.93
1451,1452,31.410000,-121.991667,0.000,33.60,0.00,7.61,322,143.1,0.731,60.46569,2.56,49.50,2280.43,2251.34
1452,1453,34.274270,-120.030570,0.517,1.58,0.35,14.47,30,319.0,0.110,249.28420,0.54,4.12,2223.88,2030.03


In [244]:
metadata = (pd.DataFrame({'columns': train_df_clean.columns})
 .merge(ocean, left_on = "columns", right_on = "Field Name", how = "left") 
 .merge(meta, left_on = "columns", right_on = "Field Name", how = "left")
)

metadata['field_name'] = metadata["Field Name_x"].fillna(metadata['Field Name_y'])
metadata['units'] = metadata["Units_x"].fillna(metadata['Units_y'])
metadata['description'] = metadata["Description_x"].fillna(metadata['Description_y'])
metadata = metadata.drop(['Description_x', 'Description_y', 'Units_x', 'Units_y', 'Field Name_x', 'Field Name_y'], axis = 1)

metadata.loc[metadata["columns"] == 'R_Oxy_micromol_Kg', "description"] = "Oxygen micromoles per kilogram seawater"
metadata.loc[metadata["columns"] == 'Salinity1', "description"] = "Reported Salinity (from Specific Volume Anomoly, M³/Kg)"
metadata.loc[metadata["columns"] == 'R_TEMP', "description"] = "Reported (Potential) Temperature in degrees Celsius"
metadata.loc[metadata["columns"] == 'Temperature_degC', "description"] = "Also temperature in degrees Celsius"
metadata.loc[metadata["columns"] == 'DIC', "description"] = "Dissolved Inorganic Carbon micromoles per kilogram solution"

metadata.loc[metadata["columns"] == 'R_Oxy_micromol_Kg', "units"] = "O2 mM/kg"
metadata.loc[metadata["columns"] == 'Salinity1', "units"] = "M³/Kg"
metadata.loc[metadata["columns"] == 'R_TEMP', "units"] = "C"
metadata.loc[metadata["columns"] == 'Temperature_degC', "units"] = "C"
metadata.loc[metadata["columns"] == 'DIC', "units"] = "mM/kg"


metadata = metadata.drop(columns = ["field_name", "Unnamed: 3"])

metadata

,columns,units,description
0,id,NaN,NaN
1,Lat_Dec,decimal degrees,Observed Latitude in decimal degrees
2,Lon_Dec,decimal degrees,Observed Longitude in decimal degrees
3,NO2uM,micromoles per liter,Micromoles Nitrite per liter of seawater
4,NO3uM,micromoles per liter,Micromoles Nitrate per liter of seawater
5,NH3uM,micromoles per liter,Micromoles Ammonia per liter of seawater
6,R_TEMP,C,Reported (Potential) Temperature in degrees Ce...
7,R_Depth,meters,Reported Depth (from pressure) in meters
8,R_Sal,Practical Salinity Scale,Reported Salinity (from Specific Volume Anomol...
9,R_DYNHT,dynamic meters,Reported Dynamic Height in units of dynamic me...


In [245]:
full_str = y_train.name + " ~ "
for col in x_train.columns:
    if col != x_train.columns[-1]:
        full_str = full_str + col + "+"
    else:
        full_str = full_str + col

full_str

'DIC ~ id+Lat_Dec+Lon_Dec+NO2uM+NO3uM+NH3uM+R_TEMP+R_Depth+R_Sal+R_DYNHT+R_Oxy_micromol_Kg+PO4uM+SiO3uM+TA1'

In [247]:
# find design matrix for regression model using 'rating' as response variable 
y_matr, x_matr = dmatrices(full_str, data=train_df_clean, return_type='dataframe')

# create DataFrame to hold VIF values
vif_df = pd.DataFrame()
vif_df['variable'] = x_matr.columns

#calculate VIF for each predictor variable 
vif_df['VIF'] = [variance_inflation_factor(x_matr.replace(0, np.nan).dropna().values, i) for i in range(x_matr.shape[1])]

vif_df

,variable,VIF
0,Intercept,123661.682964
1,id,1.023144
2,Lat_Dec,1.479910
3,Lon_Dec,1.382050
4,NO2uM,1.613419
5,NO3uM,64.739662
6,NH3uM,1.312171
7,R_TEMP,102.209103
8,R_Depth,15.667547
9,R_Sal,135.383019


In [248]:
np.percentile(train_df_clean['DIC'], 95)

2315.04

In [249]:
test_pct_counts

0       False
1       False
2       False
3       False
4       False
        ...  
1449    False
1450    False
1451    False
1452    False
1453    False
Name: DIC, Length: 1454, dtype: bool

In [250]:
test_pct_counts = (
 (train_df_clean['DIC'] > np.percentile(train_df_clean['DIC'], 99)) |
 (train_df_clean['DIC'] < np.percentile(train_df_clean['DIC'], 1))
)

test_pct_counts.where(test_pct_counts == True).dropna().value_counts().values[0]

30

In [251]:
for col in train_df_clean:
     true_counts = (
     (train_df_clean[col] > np.percentile(train_df_clean[col], 99)) |
     (train_df_clean[col] < np.percentile(train_df_clean[col], 1))
    )
    
     true_counts = true_counts.where(true_counts == True).dropna().value_counts().values[0]
    
     print(f"{col}: {np.percentile(train_df_clean[col], 0.99)}")
     print(f"\tnum outliers: {true_counts}")

id: 15.384699999999999
	num outliers: 30
Lat_Dec: 31.411088701
	num outliers: 30
Lon_Dec: -123.90777
	num outliers: 20
NO2uM: 0.0
	num outliers: 15
NO3uM: 0.0
	num outliers: 14
NH3uM: 0.0
	num outliers: 15
R_TEMP: 5.453847
	num outliers: 29
R_Depth: 2.0
	num outliers: 29
R_Sal: 108.97694
	num outliers: 30
R_DYNHT: 0.004
	num outliers: 25
R_Oxy_micromol_Kg: 6.824397263999996
	num outliers: 30
PO4uM: 0.22
	num outliers: 28
SiO3uM: 0.42
	num outliers: 28
TA1: 2207.003858
	num outliers: 30
DIC: 1989.3239709449
	num outliers: 30


In [4]:
np.percentile(train_df_clean['DIC'], 0.5)

1984.1098

In [108]:
z_scores = np.abs(stats.zscore(train_df_clean))

train_df_clean.where(z_scores > 2).dropna(how = "all").notna().sum()
# z_scores

id                     0
Lat_Dec              137
Lon_Dec               42
NO2uM                  8
NO3uM                  0
NH3uM                 53
R_TEMP                58
R_Depth               12
R_Sal                 46
R_DYNHT               12
R_Nuts                53
R_Oxy_micromol.Kg      1
PO4uM                  8
SiO3uM                27
TA1                   27
Salinity1             22
Temperature_degC      58
DIC                    0
dtype: int64

In [252]:
np.abs(stats.zscore(train_df_clean))

,id,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Oxy_micromol_Kg,PO4uM,SiO3uM,TA1,DIC
0,1.730860,1.250025,0.261255,0.113395,1.035053,0.445686,0.835676,0.372944,0.942650,0.732056,1.180854,1.098656,0.862668,0.891844,1.058135
1,1.728478,2.079771,1.036464,0.218873,1.097514,0.445686,1.016713,0.372944,0.947175,1.074428,0.884229,0.903362,0.815147,0.654648,0.916079
2,1.726095,1.250025,0.261255,0.413995,0.325198,0.445686,0.215414,0.412970,0.251955,0.631954,0.365668,0.346519,0.564713,0.717393,0.348543
3,1.723713,0.237123,1.347488,0.173166,0.748429,0.393290,0.689767,0.110973,0.746943,0.512938,0.615675,0.610421,0.342731,0.278260,0.644785
4,1.721330,2.084275,1.036079,0.218873,0.986473,0.183708,0.905929,0.372944,0.917762,1.000476,0.936008,0.864303,0.702634,0.637320,0.902996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1.721330,0.166883,1.346314,0.218873,1.368176,0.445686,1.178834,0.940068,1.202838,1.408583,1.453967,1.430655,1.534953,1.632405,1.420743
1450,1.723713,1.122787,0.112271,0.941385,0.901223,0.445686,0.628824,0.415848,0.741788,0.618259,0.847539,0.873812,0.792186,0.930159,0.897583
1451,1.726095,2.089124,1.032587,0.218873,1.021173,0.445686,0.884313,0.370065,0.921156,0.975825,0.931299,0.893597,0.710321,0.692429,0.891681
1452,1.728478,1.125709,0.108062,1.598864,1.201033,1.388155,0.969280,0.470546,1.068720,0.725079,1.112429,1.078871,0.875348,0.913967,1.064655


In [264]:
# set random forest param grid
param_rf = {
    'max_features': ('sqrt', "log2", None),
    'n_estimators': (50, 100, 200), 
    'max_depth': (3, 4, 5, 6, 7), 
    'min_samples_split': (2, 5, 10),
    'min_samples_leaf': (1, 2, 4)
}

In [265]:
# time at start of cv
time_before_rf = time.time()

# find best params for random forest
best_rf = GridSearchCV(
    RandomForestRegressor(), 
    param_rf,
    cv = 10,
    n_jobs = 5
)

best_rf.fit(x_train, y_train)

# time after cv runs
time_after_rf = time.time()

# get computation time of cv function
total_time_rf = time_after_rf - time_before_rf

In [266]:
# get total time
print(f"Total RF computation time: {round(total_time_rf/60)} minutes, {round(total_time_rf%60, 2)} seconds")

Total RF computation time: 4 minutes, 26.56 seconds


In [267]:
best_rf.best_params_

{'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 200}

In [268]:
best_rf.best_score_

0.9970835646207427

In [276]:
# set random forest param grid
param_boost_1 = {
    'learning_rate': (0.01, 0.05, 0.1),
    'n_estimators': (50, 100, 200), 
    'max_depth': (2, 3, 5), 
    'subsample': (0.6, 0.8, 1.0), 
    'min_samples_leaf': (1, 2, 4), 
    'max_features': ('sqrt', 'log2', None)
    
}

param_boost_2 = {
    'n_estimators': (50, 100, 200), 
    'max_depth': (2, 3, 5), 
    'subsample': (0.6, 0.8, 1.0), 
    'min_samples_leaf': (1, 2, 4), 
    'max_features': ('sqrt', 'log2', None)
    
}

## Step 1: find optimal number of trees

In [294]:
tree_finder = (GradientBoostingRegressor(n_estimators = 1000, 
                                        learning_rate = 0.1, 
                                        n_iter_no_change = 50) 
              )

tree_finder.fit(x_train, y_train)

GradientBoostingRegressor(n_estimators=1000, n_iter_no_change=50)

In [293]:
print(f"Most optimal tree/last tree where the model experienced improvement: tree {tree_finder.n_estimators_}")

Most optimal tree/last tree where the model experienced improvement: tree 170


## Step 2: tune learning rate

In [314]:
# make predictor grid
finder_param_grid = dict(learning_rate = uniform.rvs(size = 100, scale = 0.31, loc = 0.01))

# test parameters for optimal values
random_search_finder = RandomizedSearchCV(tree_finder, 
                                   finder_param_grid,
                                   n_iter = 20,
                                   n_jobs = -1
                                  )

# fit on best values
random_search_finder.fit(x_train, y_train)

RandomizedSearchCV(estimator=GradientBoostingRegressor(n_estimators=1000,
                                                       n_iter_no_change=50),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': array([0.12652715, 0.17126432, 0.28409987, 0.19261255, 0.13616174,
       0.04712748, 0.13845162, 0.24221554, 0.16470809, 0.11750039,
       0.08705409, 0.03115891, 0.24943557, 0.14114388, 0.18579998,
       0.1130818 , 0.23182972, 0.2355...
       0.04025063, 0.20826669, 0.04739743, 0.22562264, 0.07201508,
       0.10373111, 0.05304147, 0.30933889, 0.16514244, 0.0273647 ,
       0.07967363, 0.18852721, 0.10424001, 0.08316952, 0.17231859,
       0.27513089, 0.05279629, 0.04324343, 0.21043201, 0.25354874,
       0.16925605, 0.10637304, 0.22572541, 0.08515565, 0.28535462,
       0.28413951, 0.14959235, 0.22867317, 0.10140642, 0.1598874 ,
       0.15609451, 0.10123351, 0.21501148, 0.28831989, 0.21125468])})

In [315]:
print(f"Best learning rate: {random_search_finder.best_params_['learning_rate']}")

Best learning rate: 0.04262146566541838


## Step 3: tune tree specific parameters

In [320]:
# remake boosted model with optimal values
tree_climber = (GradientBoostingRegressor(n_estimators = tree_finder.n_estimators_, 
                                          learning_rate = random_search_finder.best_params_['learning_rate'],  
                                          n_iter_no_change = 50)
               )
                
tree_climber.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.04262146566541838, n_estimators=155,
                          n_iter_no_change=50)

In [325]:
# make new grid for new parameters
tree_param_grid = dict(max_depth = np.arange(3, 11), 
                       min_samples_leaf = np.arange(1, 11), 
                       min_impurity_decrease = uniform.rvs(size = 100, loc = 0, scale = 0.5)
                      )

# test new parameters for optimal values
random_search_climber = RandomizedSearchCV(tree_climber, 
                                            tree_param_grid,
                                            n_iter = 50,
                                            n_jobs = -1
                                  )

# fit on best values
random_search_climber.fit(x_train, y_train)

RandomizedSearchCV(estimator=GradientBoostingRegressor(learning_rate=0.04262146566541838,
                                                       n_estimators=155,
                                                       n_iter_no_change=50),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_impurity_decrease': array([0.17023461, 0.11983237, 0.01810632, 0.14751026, 0.21005976,
       0.18797963, 0.22655463, 0.34671759, 0.45374155, 0.29698503,
       0.2...
       0.23254631, 0.16870655, 0.14665635, 0.41722425, 0.0956964 ,
       0.34677671, 0.43044122, 0.04755264, 0.42815615, 0.16521641,
       0.30832946, 0.19191713, 0.1714984 , 0.29371624, 0.31815352,
       0.45335219, 0.16381989, 0.12923685, 0.30904249, 0.37014535,
       0.31198575, 0.15164433, 0.29838867, 0.25191075, 0.1694054 ,
       0.1475041 , 0.01928644, 0.21477444, 0.43058359, 0.19833546]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])})

In [327]:
print(f"Optimal minimum number of samples needed to split: {random_search_climber.best_params_['min_samples_leaf']}")
print(f"Optimal naximum tree depth: {random_search_climber.best_params_['max_depth']}")
print(f"Optimal loss reduction value: {random_search_climber.best_params_['min_impurity_decrease']}")

Optimal minimum number of samples needed to split: 10
Optimal naximum tree depth: 5
Optimal loss reduction value: 0.16200339285238158


In [332]:
# time at start of cv
time_before_boost = time.time()

# remake boosted model with additional values
tree_overlooker = GradientBoostingRegressor(**{**random_search_finder.best_params_, **random_search_climber.best_params_}, 
                                             n_iter_no_change = 50)
                   
tree_overlooker.fit(x_train, y_train)

# make param grid
overlooker_param_grid = dict(max_depth = np.arange(3, 11), 
                             min_samples_leaf = np.arange(1, 11), 
                             min_impurity_decrease = uniform.rvs(size = 100, loc = 0, scale = 0.5)
                            )

# init kfold
cv = KFold(n_splits=5, shuffle=True)


# find best boost params
best_boost = RandomizedSearchCV(
    estimator = tree_overlooker,
    param_grid = overlooker_param_grid,
    scoring='neg_root_mean_squared_error', 
    n_iter = 50
    cv=cv,
    n_jobs=-1, 
    verbose = 0
)

# Fit the model
best_boost.fit(x_train, y_train)

# time after cv runs
time_after_boost = time.time()

# get computation time of cv function
total_time_boost = time_after_boost - time_before_boost

In [333]:
# get total time
print(f"Total RF computation time: {round(total_time_boost/60)} minutes, {round(total_time_boost%60, 2)} seconds")

Total RF computation time: 18 minutes, 46.24 seconds


In [334]:
best_boost.best_params_

{'max_depth': 6,
 'min_impurity_decrease': 0.0891938717568318,
 'min_samples_leaf': 7}

In [335]:
best_boost.best_score_

-5.973601662787823

In [282]:
train_df_clean['DIC'].max() - train_df_clean['DIC'].min()

418.9500000000003

In [1]:
help(RandomizedSearchCV)

NameError: name 'RandomizedSearchCV' is not defined